Consider the rock-paper-scissors game. We model the 2-player game with a finite reward matrix $Q$:

|| Rock | Paper | Scissors |
| :-| :- | -: | :-: |
**Rock**| 0 | -1 | 1
**Paper**| 1 | 0 | -1
**Scissors**| -1 | 1 | 0

Define $x$ and $y$ as the probability associated with Player 1 and Player 2, respectively, choosing each strategy. The 2-player game optimal solution strategy can be found by solving the following:

\begin{align}
\text{min max } & \sum_{i=1}^3 \sum_{j=1}^3 x_i q_{ij}y_j\\
\text{s.t. } & \sum_{i=1}^3 x_i = 1\\
 & \sum_{j=1}^3 y_j = 1\\
 & x, y \ge 0
\end{align}

The LP formulation for the above problem is (fixing x):

\begin{align}
\text{min } & u  \\
\text{s.t. } & u \ge \sum_{i=1}^3 x_i q_{ij} && \forall j =1,\dots, 3\\
 & \sum_{i=1}^3 x_i = 1\\
 & x \ge 0
\end{align}

Solving the LP:

In [105]:
using Gurobi
using JuMP

#Washburn and Wood's sample problem

Len = 6
P = 0.1.*ones(Len)
P[1] = 0.2
P[4] = 0.2
# P = transpose(P)
println(P)
println(D)
D = [1 0 0;
    1 0 0 ;
    1 1 0;
    0 1 1;
    0 1 0;
    0 0 1]

Q = P.*D
println(Q)

gurobi_env = Gurobi.Env()
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)

println("y-problem")
m = Model(() -> Gurobi.Optimizer(gurobi_env))

# Q = [0 -1 1;
#     1 0 -1;
#     -1 1 0]
# println(Q)
@variable(m, y[1:3]>=0)
@variable(m, u)
constr = Array{JuMP.ConstraintRef}(undef, 6)
for i = 1:Len
    constr[i] = @constraint(m, u >= sum(y[j]*Q[i,j] for j = 1:3))
end
@constraint(m, sum(y[i] for i = 1:3) == 1)
@objective(m, Min, u)

optimize!(m)
println(m)
current_Obj = JuMP.objective_value.(m)
# current_x = JuMP.value.(x)
current_y = JuMP.value.(y)
current_u = JuMP.value.(u)

println(current_Obj)
# println(current_x)
println(current_y)
println(current_u)
for i = 1:Len
    println("Dual solution ",i,": ", JuMP.dual(constr[i]))
end

println("\n\n")

println("x-problem")
m1 = Model(() -> Gurobi.Optimizer(gurobi_env))

# Q = [0 -1 1;
#     1 0 -1;
#     -1 1 0]
Q = [-1 1 1; 1 -1 -1]
# println(Q)
Len = 6
# println("1")
@variable(m1, x[1:Len]>=0)
@variable(m1, w)
constr1 = Array{JuMP.ConstraintRef}(undef, 3)
for j = 1:3
    constr1[j] = @constraint(m1, w <= sum(x[i]*Q[i,j] for i = 1:Len))
end
# constr1[1] = @constraint(m1, w <= x[1]*0.4)
# constr1[2] = @constraint(m1, w <= x[2]*0.4)
# constr1[3] = @constraint(m1, w <= x[3]*0.3)
@constraint(m1, sum(x[i] for i = 1:Len) <= 1)
@objective(m1, Max, w)

optimize!(m1)
println(m1)
current_Obj = JuMP.objective_value.(m1)
current_x = JuMP.value.(x)
# current_y = JuMP.value.(y)
current_w = JuMP.value.(w)

println(current_Obj)
println(current_x)
# println(current_y)
println(current_w)
for i = 1:3
    println("Dual solution ",i,": ", JuMP.dual(constr1[i]))
end

# println(sum())

[0.2, 0.1, 0.1, 0.2, 0.1, 0.1]
[1 0 0; 1 0 0; 1 1 0; 0 1 1; 0 1 0; 0 0 1]
[0.2 0.0 0.0; 0.1 0.0 0.0; 0.1 0.1 0.0; 0.0 0.2 0.2; 0.0 0.1 0.0; 0.0 0.0 0.1]
Academic license - for non-commercial use only
y-problem
Min u
Subject to
 y[1] + y[2] + y[3] == 1.0
 -0.2 y[1] + u >= 0.0
 -0.1 y[1] + u >= 0.0
 -0.1 y[1] - 0.1 y[2] + u >= 0.0
 -0.2 y[2] - 0.2 y[3] + u >= 0.0
 -0.1 y[2] + u >= 0.0
 -0.1 y[3] + u >= 0.0
 y[1] >= 0.0
 y[2] >= 0.0
 y[3] >= 0.0

0.1
[0.5, 0.0, 0.5]
0.1
Dual solution 1: 0.5
Dual solution 2: 0.0
Dual solution 3: 0.0
Dual solution 4: 0.5
Dual solution 5: 0.0
Dual solution 6: 0.0



x-problem
Max w
Subject to
 -0.2 x[1] - 0.1 x[2] - 0.1 x[3] + w <= 0.0
 -0.1 x[3] - 0.2 x[4] - 0.1 x[5] + w <= 0.0
 -0.2 x[4] - 0.1 x[6] + w <= 0.0
 x[1] + x[2] + x[3] + x[4] + x[5] + x[6] <= 1.0
 x[1] >= 0.0
 x[2] >= 0.0
 x[3] >= 0.0
 x[4] >= 0.0
 x[5] >= 0.0
 x[6] >= 0.0

0.1
[0.5, 0.0, 0.0, 0.5, 0.0, 0.0]
0.1
Dual solution 1: -0.5
Dual solution 2: -0.0
Dual solution 3: -0.5


In [62]:
using Gurobi
using JuMP

#Washburn and Wood's sample problem
C = 5
v1 = 20
v2 = 45


# Q = [-C v v;
#     -C -C v;
#     -C v -C]

# Q =[-C v v2;
#     -C -C v2;
#     v v -C]
# arcs = [1 2; 1 3; 1 4; 2 3; 2 5; 3 5; 4 5]
# paths = [[1 2 3 5],
#         [1 2 5],
#         [1 3 5],
#         [1 4 5]]
global arcs = [1 2; 1 3; 2 3; 2 5; 2 4; 3 5; 1 4]
global paths = [[1 2 3 5],
        [1 3 5],
        [1 2 5],
        [1 2 4],
        [1 4]]
global numPaths = length(paths)
global arcNum = length(arcs[:,1])

Q = ones((length(arcs[:,1]), length(paths)))

for l = 1:length(paths)
    v = 0
    if 5 in paths[l]
        v = v1
    end
    if 4 in paths[l]
        v = v2
    end
    for k = 1:length(paths[l])-1
        arc_on_path_i = paths[l][k]
        arc_on_path_j = paths[l][k+1]
        arcSet = findall(arcs[:,1].==arc_on_path_i)
        for i in arcSet   
            if arcs[i,2] == arc_on_path_j
               Q[i,l] = -C 
            end
        end
        for i = 1:arcNum
            if Q[i,l] == 1
                Q[i,l] = v
            end
        end
    end
end
# println(Q)
# Q =[-2 -2 1 1; 
#     1 1 -2 1;
#     1 1 1 -2;
#     -2 1 1 1;
#     1 -2 1 1;
#     1 1 -2 1;
#     1 1 1 -2]

for i = 1:length(Q[:,1])
    println(Q[i,:])
end
# println(Q)
# gurobi_env = Gurobi.Env()
# setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)
m = Model(() -> Gurobi.Optimizer(gurobi_env))
set_optimizer_attribute(m, "OutputFlag", 0)


# println("y-problem")
# m = Model(() -> Gurobi.Optimizer(gurobi_env))

@variable(m, y[1:arcNum+numPaths]>=0)
@variable(m, u)
constr = Array{JuMP.ConstraintRef}(undef, arcNum+numPaths)
# for j = 1:numPaths
#     constr[j] = @constraint(m, u >= y[i]*sum(Q[i,j] for i = 1:arcNum))
# end
for k = 1:arcNum
    constr[k] = @constraint(m, u <= y[l]*sum(Q[k,l] for l = 1:numPaths))
end
# @constraint(m, sum(y[i] for i = 1:arcNum) == 1)
@constraint(m, sum(y[i] for i = 1:numPaths) == 1)
@objective(m, Max, u)

optimize!(m)
println(m)
current_Obj = JuMP.objective_value.(m)
# current_x = JuMP.value.(x)
current_y = JuMP.value.(y)
current_u = JuMP.value.(u)
current_z = zeros(arcNum)
println(current_Obj)
# println(current_x)
println(current_y[1:numPaths])
println(current_u)
# println("z = ", current_z)
for j = 1:arcNum
    current_z[j] = JuMP.dual(constr[j])
    println("Dual solution ",j,": ", current_z[j])
end

# z = zeros(4)
# z[1] = 0.375
# z[4] = 0.625
println(current_y'*Q)
println(current_y'*Q*current_z)
println(Q*current_z)

# y_temp = [1/4, 1/4, 0, 1/4, 1/4]
# println(y_temp'*Q)

[-5.0, 20.0, -5.0, -5.0, 45.0]
[20.0, -5.0, 20.0, 45.0, 45.0]
[-5.0, 20.0, 20.0, 45.0, 45.0]
[20.0, 20.0, -5.0, 45.0, 45.0]
[20.0, 20.0, 20.0, -5.0, 45.0]
[-5.0, -5.0, 20.0, 45.0, 45.0]
[20.0, 20.0, 20.0, 45.0, -5.0]
Max u
Subject to
 y[1] + y[2] + y[3] + y[4] + y[5] == 1.0
 5 y[1] - 20 y[2] + 5 y[3] + 5 y[4] - 45 y[5] + u <= 0.0
 -20 y[1] + 5 y[2] - 20 y[3] - 45 y[4] - 45 y[5] + u <= 0.0
 5 y[1] - 20 y[2] - 20 y[3] - 45 y[4] - 45 y[5] + u <= 0.0
 -20 y[1] - 20 y[2] + 5 y[3] - 45 y[4] - 45 y[5] + u <= 0.0
 -20 y[1] - 20 y[2] - 20 y[3] + 5 y[4] - 45 y[5] + u <= 0.0
 5 y[1] + 5 y[2] - 20 y[3] - 45 y[4] - 45 y[5] + u <= 0.0
 -20 y[1] - 20 y[2] - 20 y[3] - 45 y[4] + 5 y[5] + u <= 0.0
 y[1] >= 0.0
 y[2] >= 0.0
 y[3] >= 0.0
 y[4] >= 0.0
 y[5] >= 0.0
 y[6] >= 0.0
 y[7] >= 0.0
 y[8] >= 0.0
 y[9] >= 0.0
 y[10] >= 0.0
 y[11] >= 0.0
 y[12] >= 0.0

20.0
[0.0, 0.5, 0.0, 0.25, 0.25]
20.0
Dual solution 1: -0.5
Dual solution 2: -0.0
Dual solution 3: -0.0
Dual solution 4: -0.0
Dual solution 5: -0.0
Dua

DimensionMismatch: DimensionMismatch("second dimension of A, 7, does not match length of x, 12")

In [100]:
using Gurobi
using JuMP

#Solve as a max-flow problem
#Washburn and Wood's sample problem
C = -2 #100
v = 10 #50
# v2 = 20
Len = 3


arcs = [1 2; 1 3; 1 4; 2 3; 2 5; 3 5; 4 5]

N = collect(1:maximum(arcs))
origin = 1
destination = 5
arcNum = length(arcs[:,1])

x = zeros(arcNum)
# x[4] = 1

gurobi_env = Gurobi.Env()
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)

println("y-problem")
m = Model(() -> Gurobi.Optimizer(gurobi_env))

@variable(m, y[1:length(Q[:,1])]>=0)
@variable(m, u)
total_constr_num = length(N)-1 + arcNum
constr = Array{JuMP.ConstraintRef}(undef, total_constr_num)
conNum = 1
#Flow constraints:
constr[conNum] = @constraint(m, sum(y[i] for i in findall(arcs[:,1].==origin)) == 1)
conNum = conNum + 1
for node in N
    if node != origin && node !=destination
        incoming = findall(arcs[:,2].==node)
        outgoing = findall(arcs[:,1].==node)
        constr[conNum] = @constraint(m, sum(y[i] for i in outgoing)-sum(y[i] for i in incoming) == 0)
        conNum = conNum + 1
    end
end
#Capacity Constraints
# for i = 1:arcNum
#     constr[conNum] = @constraint(m,  y[i] <= (v+C)*(1-x[i]))
#     conNum = conNum + 1
# end
for i = 1:arcNum
    constr[conNum] = @constraint(m,  y[i] <= u - x[i])
    conNum = conNum + 1
end

@objective(m, Min, u)
optimize!(m)
println(termination_status(m))
println(m)
current_Obj = JuMP.objective_value.(m)
# current_x = JuMP.value.(x)
current_y = JuMP.value.(y)
current_u = JuMP.value.(u)
current_z = zeros(total_constr_num)
println(current_Obj)
# println(current_x)
println(current_y)
println(current_u)
println("z = ", current_z)
println("total constr = ", total_constr_num)
for j = 1:total_constr_num
    current_z[j] = JuMP.dual(constr[j])
#     println("Dual solution ",j,": ", current_z[j])
end
println("Dual solution : ", current_z)

print("\nConstraint to be added to the MP: ", current_z[1],)
for i = length(N)-1:total_constr_num
    print(" + ", -current_z[i],"x[i]")
end

Academic license - for non-commercial use only
y-problem
OPTIMAL
Min u
Subject to
 y[1] + y[2] + y[3] == 1.0
 -y[1] + y[4] + y[5] == 0.0
 -y[2] - y[4] + y[6] == 0.0
 -y[3] + y[7] == 0.0
 y[1] - u <= 0.0
 y[2] - u <= 0.0
 y[3] - u <= 0.0
 y[4] - u <= 0.0
 y[5] - u <= 0.0
 y[6] - u <= 0.0
 y[7] - u <= 0.0
 y[1] >= 0.0
 y[2] >= 0.0
 y[3] >= 0.0
 y[4] >= 0.0
 y[5] >= 0.0
 y[6] >= 0.0
 y[7] >= 0.0

0.3333333333333333
[0.333333, 0.333333, 0.333333, 0.0, 0.333333, 0.333333, 0.333333]
0.3333333333333333
z = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
total constr = 11
Dual solution : [0.333333, 0.333333, 0.333333, 0.0, 0.0, 0.0, -0.333333, 0.0, -0.333333, -0.333333, 0.0]

Constraint to be added to the MP: 0.3333333333333333 + -0.0x[i] + -0.0x[i] + -0.0x[i] + 0.3333333333333333x[i] + -0.0x[i] + 0.3333333333333333x[i] + 0.3333333333333333x[i] + -0.0x[i]

In [83]:
# current_y'*Q*current_z
println(current_y)
println(Q)
println(current_z)
current_y'*Q

[0.333333, 0.333333, 0.333333, 0.0, 0.333333, 0.333333, 0.333333]
[-2.0 -2.0 10.0 10.0; 10.0 10.0 -2.0 10.0; 10.0 10.0 10.0 -2.0; -2.0 10.0 10.0 10.0; 10.0 -2.0 10.0 10.0; -2.0 10.0 -2.0 10.0; 10.0 10.0 10.0 -2.0]
[0.333333, 0.333333, 0.333333, 0.0, 0.0, 0.0, -0.333333, 0.0, -0.333333, -0.333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


1×4 LinearAlgebra.Adjoint{Float64,Array{Float64,1}}:
 12.0  12.0  12.0  12.0

In [11]:
println("\n\n")

println("z-problem")
m1 = Model(() -> Gurobi.Optimizer(gurobi_env))

# Q = [0 -1 1;
#     1 0 -1;
#     -1 1 0]
# println(Q)
Len = 3

@variable(m1, z[1:Len]>=0)
@variable(m1, w)
constr1 = Array{JuMP.ConstraintRef}(undef, Len)
# for i = 1:Len
#     constr1[i] = @constraint(m1, w <= x[i]*sum(P[i]*D[i,j] for j = 1:3))
# end
constr1[1] = @constraint(m1, w <= -C*z[1] + v*z[2] + v2*z[3])
constr1[2] = @constraint(m1, w <= -C*z[1] - C*z[2] + v2*z[3])
constr1[3] = @constraint(m1, w <= v*z[1] + v*z[2] - C*z[3])
constr_w = @constraint(m1, sum(z[i] for i = 1:Len) == 1)
@objective(m1, Max, w)

optimize!(m1)
println(m1)
current_Obj = JuMP.objective_value.(m1)
current_z = JuMP.value.(z)
# current_y = JuMP.value.(y)
current_w = JuMP.value.(w)

println(current_Obj)
println(current_z)
# println(current_y)
println(current_w)
for i = 1:Len
    println("Dual solution ",i,": ", JuMP.dual(constr1[i]))
end
println("Dual solution -- last : ", JuMP.dual(constr_w))
# println(sum())

Academic license - for non-commercial use only
y-problem
Min u
Subject to
 y[1] + y[2] + y[3] + y[4] + y[5] + y[6] + y[7] + y[8] + y[9] + y[10] == 1.0
 2 y[1] - y[2] - y[3] + 2 y[4] - y[5] - y[6] - y[7] + 2 y[8] - y[9] - y[10] + u >= 0.0
 2 y[1] - y[2] - y[3] - y[4] + 2 y[5] - y[6] - y[7] - y[8] - y[9] - y[10] + u >= 0.0
 -y[1] + 2 y[2] - y[3] - y[4] - y[5] + 2 y[6] - y[7] - y[8] - y[9] + 2 y[10] + u >= 0.0
 -y[1] - y[2] + 2 y[3] - y[4] - y[5] - y[6] + 2 y[7] - y[8] + 2 y[9] - y[10] + u >= 0.0
 y[1] >= 0.0
 y[2] >= 0.0
 y[3] >= 0.0
 y[4] >= 0.0
 y[5] >= 0.0
 y[6] >= 0.0
 y[7] >= 0.0
 y[8] >= 0.0
 y[9] >= 0.0
 y[10] >= 0.0

0.0
[0.333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333333, 0.0, 0.0, 0.333333]
0.0
Dual solution 1: 0.3333333333333333
Dual solution 2: -0.0
Dual solution 3: 0.3333333333333333
Dual solution 4: 0.3333333333333334


UndefRefError: UndefRefError: access to undefined reference

[0.0, 0.0, 1.0, 1.0]
[3.0, 1.0, 0.0, 1.0]
[3.0, 1.0, 1.0, 0.0]
[0.0, 1.0, 1.0, 1.0]
[3.0, 0.0, 1.0, 1.0]
[3.0, 1.0, 0.0, 1.0]


BoundsError: BoundsError: attempt to access 6×4 Array{Float64,2} at index [7, Base.Slice(Base.OneTo(4))]

In [86]:
num = 10000000
E = 0
for i = 1:num
    y_rnd = rand()
    z_rnd = rand()
    
    if y_rnd <= 0.05
        y = 2
    else
        if y_rnd <= 1
            y = 3
        else
            y = 1
        end
    end
    
    if z_rnd <= 1
        z = 2
    else
        if z_rnd <=1
            z = 3
        else
            z = 1
        end
    end
    r = Q[y,z]
    E = E + r
end
       
println(E/num)

42.512435
